THIS NOTEBOOK IS FOR A DATA SCIENCE CAPSTONE PROJECT

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
print("week 2")

week 2


SEGMENTING AND CLUSTERING THE NEIGHBOURHOODS IN TORONTO

Import the necessary libraries

In [12]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import csv

WEBSCRAPPING

In [13]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

GET COLUMNS AND ROWS

In [14]:
headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

In [15]:
with open('Toronto.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

LOAD CSV BACK INTO A PANDAS DATAFRAME

In [16]:
df = pd.read_csv('Toronto.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [17]:

df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [18]:
df.describe()

,Postcode,Borough,Neighbourhood
count,289,289,289
unique,180,12,210
top,M9V,Not assigned,Not assigned\n
freq,8,77,78


In [19]:
df.shape

(289, 3)

DATA CLEANING

rename columns 

In [20]:
df.rename(columns={'Neighbourhood\n':'Neighborhood','Postcode':'PostalCode'}, inplace=True)
df.head()
# Change Postcode to PostalCode

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [21]:
df['Neighborhood']=df['Neighborhood'].replace(to_replace='\n', value='', regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


remove data with 'NOT ASSIGNED' borough

In [22]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [23]:
df.shape

(212, 3)

Combining neighbourhoods into one postal code area

Test with PostalCode "M5A"

In [24]:
df.query('PostalCode == "M5A"')

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


Group the neighbourhoods

In [25]:

df_new=df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [26]:
df_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [27]:
df_new.query('Neighborhood == "Not assigned"')

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [28]:
#Replace "Not assigned" with the Borough name
df_new.loc[df_new.Neighborhood == 'Not assigned', "Neighborhood"] = df_new.Borough

In [29]:
df_new.query('PostalCode == "M7A"')

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [30]:
# Data was successfully replaced

Lastly, dispalce the number of rows

In [31]:
df_new.shape

(103, 3)

Write the new dataframe to a new CSV

In [32]:
df_new.to_csv('Toronto_New.csv')